# Agent with RAG.

## Install the requirements

In [ ]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3

## 将./data/data.md 文件 存储到向量数据库

In [13]:
# 引入依赖包
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import PromptTemplate
from string import Template
import uuid
import os
from dotenv import load_dotenv

In [12]:
# 读取 ./data/data.md文件
file_path = os.path.join('data','data.md')
with open(file_path, 'r' , encoding="utf8") as file:
    doc_content = file.read()

# split the document into chunks base on markdown headers

headers_to_splits = [
    ("#","Header 1"),
    ("##","Header 2"),
    ("###","Header 3"),
]    

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_splits)

chunks = splitter.split_text(doc_content)

print("Split chunks size:", len(chunks),"/n")

for chunk in chunks :
    print(chunk)

    

Split chunks size: 11 /n
page_content='- **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。' metadata={'Header 1': '支付系统运维知识库', 'Header 2': '1. 系统监控指标'}
page_content='- **问题描述**: 用户发起支付后，交易未能成功完成。
- **可能原因**:
- 网络延迟或中断
- 支付网关服务异常
- **解决办法**:
- 检查网络连接，确保支付服务的网络畅通。
- 检查支付网关服务状态，重启服务或联系服务提供商。' metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.1 交易失败'}
page_content='- **问题描述**: 用户支付请求处理时间超过正常范围。
- **可能原因**:
- 系统资源不足
- 数据库查询效率低下
- 外部服务响应慢
- **解决办法**:
- 增加系统资源，如CPU、内存。
- 优化数据库查询，使用索引，减少复杂查询。
- 与外部服务提供商沟通，优化接口性能。' metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.2 响应时间过长'}
page_content='- **问题描述**: 支付系统完全无法访问或服务中断。
- **可能原因**:
- 主机硬件故障
- 系统软件崩溃
- 网络设备故障
- **解决办法**:
- 快速切换到备用服务器。
- 检查系统日志，定位问题原因。
- 联系硬件供应商，进行故障排查和修复。' metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.3 系统宕机'}
page_content='- **问题描述**: 检测到异常交易或系统遭受攻击。
- **可能原因**:
- 账户被盗用
- 系统存在安全漏洞
- DDoS攻击
- **解决办法**:

In [16]:
# 加载环境变量
load_dotenv()

embedings_path = os.path.join("embedings",str(uuid.uuid4))

embeding_provider = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Chroma.from_documents(documents=chunks,embedding= embeding_provider, persist_directory= embedings_path)

vector_store.persist()



/var/folders/38/9vbj12ds0z7gngdmvr90qnfh0000gn/T/ipykernel_24882/989420910.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [21]:
# 传统RAG 流程

retriever = vector_store.as_retriever()

template = """
使用以下上下文来回答最后的问题。
如果你不知道答案，就说不知道，不要试图编造答案。
最多使用三句话，并尽可能简洁地回答。
在答案的最后一定要说“谢谢询问！”。

Context: {context}

Question: {question}

Helpful Answer:"""

prompt = PromptTemplate.from_template(template)


def format_docs (docs):
    print("匹配到的知识库片段：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)


llm = ChatOpenAI(model="gpt-4o")

rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


res = rag_chain.invoke("谁管的系统最多？")

print("LLM的回答:\n",res)



匹配到的知识库片段：
 - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
- **负载均衡**: 使用负载均衡技术，分散请求压力。

- **发现问题**: 监控系统发现异常指标。
- **快速响应**: 立即通知运维团队，进行初步诊断。
- **问题定位**: 通过日志分析，定位问题原因。
- **解决方案**: 根据问题类型，执行相应的解决方案。
- **问题解决**: 确认问题解决，恢复正常服务。
- **后续跟进**: 记录问题处理过程，进行事后分析和总结。
LLM的回答:
 不知道。谢谢询问！


In [29]:
# 反馈(Reflection) 模式


# 根据问题去向量数据库查询相关的chunks
def search_in_vector_store(query: str, k=1):
    chunks = vector_store.similarity_search_with_score(query=query, k=k)
    return "\n\n".join(chunk.page_content for chunk, score in chunks)


# 询问大模型根据以上的chunk 是否能回答问题，如果能回答退出循环，如果不能继续添加chunk

# 询问是否能根据查询到的chunk 回答问题的系统提示语
prompt_can_answer_by_chunks_str = """
请根据我提供的上下文判断是否能回答用户提出的问题，如果能请返回结果can_answer,否则返回cannot_answer。
Context: \n {context}
Question: {question}
"""

prompt_can_answer_by_chunks = PromptTemplate.from_template(
    prompt_can_answer_by_chunks_str
)

ref_rag_chain = prompt_can_answer_by_chunks | llm | StrOutputParser()

k = 1
all_chunks = ""
question = "谁管的系统最多？"

while True:
    if k > 15:
        break
    else:
        k += 1

    all_chunks += search_in_vector_store(question, k) + "\n"

    res = ref_rag_chain.invoke({"context": all_chunks, "question": question})

    print(f"LLM第{k}次检查结果:{res} \n\n")
    if res == "can_answer":
        break

print(f"查询到的chunks为：\n{all_chunks}\n")

# 开始调用LLM最终回答问题

prompt_answer_final_question_str = """
请根据我提供的上下文判断是否能回答用户提出的问题
Context:\n{context}
Question: {question}
"""

prompt_answer_final_question = PromptTemplate.from_template(
    prompt_answer_final_question_str
)


final_rag_chain = prompt_answer_final_question | llm | StrOutputParser()


res = final_rag_chain.invoke({"context": all_chunks, "question": question})


print(f"LLM的最终答案为：{res}")

LLM第2次检查结果:cannot_answer 


LLM第3次检查结果:cannot_answer 


LLM第4次检查结果:cannot_answer 


LLM第5次检查结果:cannot_answer 


LLM第6次检查结果:cannot_answer 


LLM第7次检查结果:cannot_answer 


LLM第8次检查结果:cannot_answer 


LLM第9次检查结果:can_answer 


查询到的chunks为：
- **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。
- **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
- **负载均衡**: 使用负载均衡技术，分散请求压力。
- **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
- **负载均衡**: 使用负载均衡技术，分散请求压力。

- **发现问题*